In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import swifter

from pyensembl import EnsemblRelease
from pyensembl.genome import Genome
from pyensembl.shell import collect_all_installed_ensembl_releases
collect_all_installed_ensembl_releases()

/cluster/project/beltrao/gankin/software/miniconda3/envs/vnn/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[EnsemblRelease(release=75, species='homo_sapiens'),
 EnsemblRelease(release=111, species='homo_sapiens')]

# Load UKB BIM

In [2]:
data_dir = "/cluster/project/beltrao/gankin/vnn/data"

# Read the .bim file to get SNP information
bim_file = data_dir + "/ukb.bim"
bim_df = pd.read_csv(bim_file, sep='\t', header=None)
bim_df.columns = ['chrom', 'snp', 'cm', 'pos', 'a1', 'a2']
bim_df

/scratch/tmp.20761925.dgankin/ipykernel_3600543/1028760957.py:5: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  bim_df = pd.read_csv(bim_file, sep='\t', header=None)


,chrom,snp,cm,pos,a1,a2
0,1,rs28659788,0,723307,C,G
1,1,rs116587930,0,727841,G,A
2,1,rs116720794,0,729632,C,T
3,1,rs3131972,0,752721,A,G
4,1,rs12184325,0,754105,C,T
...,...,...,...,...,...,...
805421,MT,Affx-92047842,0,16337,C,T
805422,MT,Affx-79443531,0,16356,T,C
805423,MT,Affx-79443532,0,16362,T,C
805424,MT,Affx-89025709,0,16390,G,A


# Load Open Targets data

In [3]:
variant_index_parquet = data_dir + "/complete_variant_index.parquet"
variant_index = pd.read_parquet(variant_index_parquet) # takes a few minutes
variant_index

,chr_id,position,ref_allele,alt_allele,chr_id_b37,position_b37,rs_id,most_severe_consequence,cadd,af,gene_id_any_distance,gene_id_any,gene_id_prot_coding_distance,gene_id_prot_coding
0,1,30794,G,T,1,30794,rs1311411458,intron_variant,"{'phred': 0.177, 'raw': -0.402967}","{'gnomad_afr': 0.0, 'gnomad_amr': 0.0, 'gnomad...",34625.0,ENSG00000186092,34625.0,ENSG00000186092
1,1,55330,G,A,1,55330,rs185215913,downstream_gene_variant,"{'phred': 0.431, 'raw': -0.275244}","{'gnomad_afr': 0.016711590296495958, 'gnomad_a...",10089.0,ENSG00000186092,10089.0,ENSG00000186092
2,1,57248,C,T,1,57248,rs1249676487,downstream_gene_variant,"{'phred': 7.268, 'raw': 0.304146}","{'gnomad_afr': 0.0, 'gnomad_amr': 0.0, 'gnomad...",8171.0,ENSG00000186092,8171.0,ENSG00000186092
3,1,66381,T,A,1,66381,rs538833530,upstream_gene_variant,"{'phred': 1.389, 'raw': -0.101295}","{'gnomad_afr': 0.15321011673151752, 'gnomad_am...",962.0,ENSG00000186092,962.0,ENSG00000186092
4,1,67631,G,C,1,67631,rs533896527,upstream_gene_variant,"{'phred': 0.033, 'raw': -0.632235}","{'gnomad_afr': 0.041812315138478086, 'gnomad_a...",2212.0,ENSG00000186092,2212.0,ENSG00000186092
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72878704,X,156006466,G,C,X,155236131,rs1218662062,intron_variant,"{'phred': 1.118, 'raw': -0.133403}","{'gnomad_afr': 0.0, 'gnomad_amr': 0.0, 'gnomad...",8770.0,ENSG00000124334,8770.0,ENSG00000124334
72878705,X,156006493,C,T,X,155236158,rs1417497789,intron_variant,"{'phred': 1.776, 'raw': -0.063666}","{'gnomad_afr': 0.0, 'gnomad_amr': 0.0011848341...",8797.0,ENSG00000124334,8797.0,ENSG00000124334
72878706,X,156008488,C,T,X,155238153,rs186430584,intron_variant,"{'phred': 3.876, 'raw': 0.076619}","{'gnomad_afr': 0.030184331797235023, 'gnomad_a...",10792.0,ENSG00000124334,10792.0,ENSG00000124334
72878707,X,156009035,CTG,C,X,155238700,rs1381721936,intron_variant,"{'phred': None, 'raw': None}","{'gnomad_afr': 0.0008291873963515755, 'gnomad_...",11339.0,ENSG00000124334,11339.0,ENSG00000124334


# Merge with UKB SNP data

In [4]:
# merge bim with variant index
ot_bim = variant_index.merge(bim_df, left_on='rs_id', right_on='snp', how='inner')
ot_bim

,chr_id,position,ref_allele,alt_allele,chr_id_b37,position_b37,rs_id,most_severe_consequence,cadd,af,gene_id_any_distance,gene_id_any,gene_id_prot_coding_distance,gene_id_prot_coding,chrom,snp,cm,pos,a1,a2
0,1,1169204,T,C,1,1104584,rs112695918,upstream_gene_variant,"{'phred': 0.61, 'raw': -0.223786}","{'gnomad_afr': 0.1370131366674349, 'gnomad_amr...",4676.0,ENSG00000162571,4676.0,ENSG00000162571,1,rs112695918,0,1104584,T,C
1,1,2038313,C,T,1,1969752,rs74372554,intergenic_variant,"{'phred': 0.309, 'raw': -0.324059}","{'gnomad_afr': 0.01078971533516988, 'gnomad_am...",12098.0,ENSG00000067606,12098.0,ENSG00000067606,1,rs74372554,0,1969752,C,T
2,1,2194104,A,G,1,2125543,rs146607591,missense_variant,"{'phred': 4.241, 'raw': 0.098415}","{'gnomad_afr': 0.00034498620055197794, 'gnomad...",18616.0,ENSG00000162585,18616.0,ENSG00000162585,1,rs146607591,0,2125543,A,G
3,1,2592668,C,T,1,2524107,rs146100312,missense_variant,"{'phred': 23.9, 'raw': 3.118097}","{'gnomad_afr': 0.0, 'gnomad_amr': 0.0, 'gnomad...",6177.0,ENSG00000157870,6177.0,ENSG00000157870,1,rs146100312,0,2524107,C,T
4,1,2828773,C,A,1,2745338,rs116076619,intergenic_variant,"{'phred': 0.058, 'raw': -0.555543}","{'gnomad_afr': 0.0060821666284140465, 'gnomad_...",27080.0,ENSG00000215912,27080.0,ENSG00000215912,1,rs116076619,0,2745338,C,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
747754,X,152014563,G,A,X,151183035,rs145754054,intergenic_variant,"{'phred': 2.456, 'raw': -0.011208}","{'gnomad_afr': 0.009298393913778529, 'gnomad_a...",39883.0,ENSG00000102287,39883.0,ENSG00000102287,X,rs145754054,0,151183035,G,A
747755,X,153459591,C,T,X,152725049,rs5945389,intron_variant,"{'phred': 2.968, 'raw': 0.022059}","{'gnomad_afr': 0.0032312925170068026, 'gnomad_...",10996.0,ENSG00000183479,10996.0,ENSG00000183479,X,rs5945389,0,152725049,C,T
747756,X,153781540,A,C,X,153046995,rs199915647,missense_variant,"{'phred': 22.5, 'raw': 2.480113}","{'gnomad_afr': 0.00017018379850238256, 'gnomad...",5128.0,ENSG00000184343,5128.0,ENSG00000184343,X,rs199915647,0,153046995,A,C
747757,X,154534419,G,A,X,153762634,rs5030868,missense_variant,"{'phred': 24.7, 'raw': 3.374046}","{'gnomad_afr': 0.0001721763085399449, 'gnomad_...",6780.0,ENSG00000269335,6780.0,ENSG00000269335,X,rs5030868,0,153762634,G,A


# Gene ids to gene names

In [6]:
human_data = EnsemblRelease(species='human', release=75)
def get_gene_name(ensembl_id):
    try:
        return human_data.gene_name_of_gene_id(ensembl_id)
    except:
        return None

In [9]:
name_map = pd.DataFrame()
name_map["gene_id"] = ot_bim.gene_id_prot_coding.unique()
name_map["gene_name"]  = name_map["gene_id"].swifter.apply(lambda x: get_gene_name(x))

Pandas Apply: 100%|██████████| 18817/18817 [10:42<00:00, 29.27it/s]


In [11]:
# count None in gene names
print("Gene Id not mappable: ", name_map.gene_name.isnull().sum())
print("Unique gene names: ", name_map.gene_name.nunique())

Gene Id not mappable:  298
Unique gene names:  18510


In [13]:
ot_bim = pd.merge(ot_bim, name_map, left_on='gene_id_prot_coding', right_on='gene_id', how='left')

In [14]:
# remove all None gene names
ot_bim = ot_bim[ot_bim.gene_name.notnull()]
ot_bim

,chr_id,position,ref_allele,alt_allele,chr_id_b37,position_b37,rs_id,most_severe_consequence,cadd,af,...,gene_id_prot_coding_distance,gene_id_prot_coding,chrom,snp,cm,pos,a1,a2,gene_id,gene_name
0,1,1169204,T,C,1,1104584,rs112695918,upstream_gene_variant,"{'phred': 0.61, 'raw': -0.223786}","{'gnomad_afr': 0.1370131366674349, 'gnomad_amr...",...,4676.0,ENSG00000162571,1,rs112695918,0,1104584,T,C,ENSG00000162571,TTLL10
1,1,2038313,C,T,1,1969752,rs74372554,intergenic_variant,"{'phred': 0.309, 'raw': -0.324059}","{'gnomad_afr': 0.01078971533516988, 'gnomad_am...",...,12098.0,ENSG00000067606,1,rs74372554,0,1969752,C,T,ENSG00000067606,PRKCZ
2,1,2194104,A,G,1,2125543,rs146607591,missense_variant,"{'phred': 4.241, 'raw': 0.098415}","{'gnomad_afr': 0.00034498620055197794, 'gnomad...",...,18616.0,ENSG00000162585,1,rs146607591,0,2125543,A,G,ENSG00000162585,C1orf86
3,1,2592668,C,T,1,2524107,rs146100312,missense_variant,"{'phred': 23.9, 'raw': 3.118097}","{'gnomad_afr': 0.0, 'gnomad_amr': 0.0, 'gnomad...",...,6177.0,ENSG00000157870,1,rs146100312,0,2524107,C,T,ENSG00000157870,FAM213B
4,1,2828773,C,A,1,2745338,rs116076619,intergenic_variant,"{'phred': 0.058, 'raw': -0.555543}","{'gnomad_afr': 0.0060821666284140465, 'gnomad_...",...,27080.0,ENSG00000215912,1,rs116076619,0,2745338,C,A,ENSG00000215912,TTC34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
747754,X,152014563,G,A,X,151183035,rs145754054,intergenic_variant,"{'phred': 2.456, 'raw': -0.011208}","{'gnomad_afr': 0.009298393913778529, 'gnomad_a...",...,39883.0,ENSG00000102287,X,rs145754054,0,151183035,G,A,ENSG00000102287,GABRE
747755,X,153459591,C,T,X,152725049,rs5945389,intron_variant,"{'phred': 2.968, 'raw': 0.022059}","{'gnomad_afr': 0.0032312925170068026, 'gnomad_...",...,10996.0,ENSG00000183479,X,rs5945389,0,152725049,C,T,ENSG00000183479,TREX2
747756,X,153781540,A,C,X,153046995,rs199915647,missense_variant,"{'phred': 22.5, 'raw': 2.480113}","{'gnomad_afr': 0.00017018379850238256, 'gnomad...",...,5128.0,ENSG00000184343,X,rs199915647,0,153046995,A,C,ENSG00000184343,SRPK3
747757,X,154534419,G,A,X,153762634,rs5030868,missense_variant,"{'phred': 24.7, 'raw': 3.374046}","{'gnomad_afr': 0.0001721763085399449, 'gnomad_...",...,6780.0,ENSG00000269335,X,rs5030868,0,153762634,G,A,ENSG00000269335,IKBKG


In [15]:
ot_bim.rs_id.nunique()

731084

In [16]:
ot_bim = ot_bim.drop_duplicates(subset=["rs_id", "gene_name"])
# rename rs_id col to snp
ot_bim = ot_bim.rename({"rs_id":"snp"})
ot_bim

,chr_id,position,ref_allele,alt_allele,chr_id_b37,position_b37,rs_id,most_severe_consequence,cadd,af,...,gene_id_prot_coding_distance,gene_id_prot_coding,chrom,snp,cm,pos,a1,a2,gene_id,gene_name
0,1,1169204,T,C,1,1104584,rs112695918,upstream_gene_variant,"{'phred': 0.61, 'raw': -0.223786}","{'gnomad_afr': 0.1370131366674349, 'gnomad_amr...",...,4676.0,ENSG00000162571,1,rs112695918,0,1104584,T,C,ENSG00000162571,TTLL10
1,1,2038313,C,T,1,1969752,rs74372554,intergenic_variant,"{'phred': 0.309, 'raw': -0.324059}","{'gnomad_afr': 0.01078971533516988, 'gnomad_am...",...,12098.0,ENSG00000067606,1,rs74372554,0,1969752,C,T,ENSG00000067606,PRKCZ
2,1,2194104,A,G,1,2125543,rs146607591,missense_variant,"{'phred': 4.241, 'raw': 0.098415}","{'gnomad_afr': 0.00034498620055197794, 'gnomad...",...,18616.0,ENSG00000162585,1,rs146607591,0,2125543,A,G,ENSG00000162585,C1orf86
3,1,2592668,C,T,1,2524107,rs146100312,missense_variant,"{'phred': 23.9, 'raw': 3.118097}","{'gnomad_afr': 0.0, 'gnomad_amr': 0.0, 'gnomad...",...,6177.0,ENSG00000157870,1,rs146100312,0,2524107,C,T,ENSG00000157870,FAM213B
4,1,2828773,C,A,1,2745338,rs116076619,intergenic_variant,"{'phred': 0.058, 'raw': -0.555543}","{'gnomad_afr': 0.0060821666284140465, 'gnomad_...",...,27080.0,ENSG00000215912,1,rs116076619,0,2745338,C,A,ENSG00000215912,TTC34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
747753,X,151609900,A,G,X,150778372,rs58448897,intron_variant,"{'phred': 3.523, 'raw': 0.055668}","{'gnomad_afr': 0.11482740118046861, 'gnomad_am...",...,46225.0,ENSG00000166049,X,rs58448897,0,150778372,A,G,ENSG00000166049,PASD1
747754,X,152014563,G,A,X,151183035,rs145754054,intergenic_variant,"{'phred': 2.456, 'raw': -0.011208}","{'gnomad_afr': 0.009298393913778529, 'gnomad_a...",...,39883.0,ENSG00000102287,X,rs145754054,0,151183035,G,A,ENSG00000102287,GABRE
747755,X,153459591,C,T,X,152725049,rs5945389,intron_variant,"{'phred': 2.968, 'raw': 0.022059}","{'gnomad_afr': 0.0032312925170068026, 'gnomad_...",...,10996.0,ENSG00000183479,X,rs5945389,0,152725049,C,T,ENSG00000183479,TREX2
747756,X,153781540,A,C,X,153046995,rs199915647,missense_variant,"{'phred': 22.5, 'raw': 2.480113}","{'gnomad_afr': 0.00017018379850238256, 'gnomad...",...,5128.0,ENSG00000184343,X,rs199915647,0,153046995,A,C,ENSG00000184343,SRPK3


In [18]:
# get snp_bed_id from bim_df 
bim_df = bim_df.reset_index()
#bim_df.columns = ['snp_bed_id', 'chrom', 'snp', 'cm', 'pos', 'a1', 'a2']
bim_df.columns = ['snp_bed_id', 'chrom', 'snp', 'cm', 'pos', 'a1', 'a2']
bim_df

,snp_bed_id,chrom,snp,a1,a2,maf,nchrobs
0,0,1,rs28659788,0,723307,C,G
1,1,1,rs116587930,0,727841,G,A
2,2,1,rs116720794,0,729632,C,T
3,3,1,rs3131972,0,752721,A,G
4,4,1,rs12184325,0,754105,C,T
...,...,...,...,...,...,...,...
805421,805421,MT,Affx-92047842,0,16337,C,T
805422,805422,MT,Affx-79443531,0,16356,T,C
805423,805423,MT,Affx-79443532,0,16362,T,C
805424,805424,MT,Affx-89025709,0,16390,G,A


In [19]:
ot_bim = ot_bim.merge(bim_df[["snp", "snp_bed_id"]], left_on="snp", right_on="snp")

In [20]:
ot_bim

,chr_id,position,ref_allele,alt_allele,chr_id_b37,position_b37,rs_id,most_severe_consequence,cadd,af,...,gene_id_prot_coding,chrom,snp,cm,pos,a1,a2,gene_id,gene_name,snp_bed_id
0,1,1169204,T,C,1,1104584,rs112695918,upstream_gene_variant,"{'phred': 0.61, 'raw': -0.223786}","{'gnomad_afr': 0.1370131366674349, 'gnomad_amr...",...,ENSG00000162571,1,rs112695918,0,1104584,T,C,ENSG00000162571,TTLL10,220
1,1,2038313,C,T,1,1969752,rs74372554,intergenic_variant,"{'phred': 0.309, 'raw': -0.324059}","{'gnomad_afr': 0.01078971533516988, 'gnomad_am...",...,ENSG00000067606,1,rs74372554,0,1969752,C,T,ENSG00000067606,PRKCZ,599
2,1,2194104,A,G,1,2125543,rs146607591,missense_variant,"{'phred': 4.241, 'raw': 0.098415}","{'gnomad_afr': 0.00034498620055197794, 'gnomad...",...,ENSG00000162585,1,rs146607591,0,2125543,A,G,ENSG00000162585,C1orf86,648
3,1,2592668,C,T,1,2524107,rs146100312,missense_variant,"{'phred': 23.9, 'raw': 3.118097}","{'gnomad_afr': 0.0, 'gnomad_amr': 0.0, 'gnomad...",...,ENSG00000157870,1,rs146100312,0,2524107,C,T,ENSG00000157870,FAM213B,838
4,1,2828773,C,A,1,2745338,rs116076619,intergenic_variant,"{'phred': 0.058, 'raw': -0.555543}","{'gnomad_afr': 0.0060821666284140465, 'gnomad_...",...,ENSG00000215912,1,rs116076619,0,2745338,C,A,ENSG00000215912,TTC34,889
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
731079,X,151609900,A,G,X,150778372,rs58448897,intron_variant,"{'phred': 3.523, 'raw': 0.055668}","{'gnomad_afr': 0.11482740118046861, 'gnomad_am...",...,ENSG00000166049,X,rs58448897,0,150778372,A,G,ENSG00000166049,PASD1,802181
731080,X,152014563,G,A,X,151183035,rs145754054,intergenic_variant,"{'phred': 2.456, 'raw': -0.011208}","{'gnomad_afr': 0.009298393913778529, 'gnomad_a...",...,ENSG00000102287,X,rs145754054,0,151183035,G,A,ENSG00000102287,GABRE,802336
731081,X,153459591,C,T,X,152725049,rs5945389,intron_variant,"{'phred': 2.968, 'raw': 0.022059}","{'gnomad_afr': 0.0032312925170068026, 'gnomad_...",...,ENSG00000183479,X,rs5945389,0,152725049,C,T,ENSG00000183479,TREX2,802547
731082,X,153781540,A,C,X,153046995,rs199915647,missense_variant,"{'phred': 22.5, 'raw': 2.480113}","{'gnomad_afr': 0.00017018379850238256, 'gnomad...",...,ENSG00000184343,X,rs199915647,0,153046995,A,C,ENSG00000184343,SRPK3,802648


# Load GO term to gene mapping

In [2]:
import networkx as nx
import pickle 

In [4]:
go_path = "../../data/GO_biological_process.pickle"
G = pickle.load(open(go_path, 'rb'))

In [5]:
# Load go terms and genes
goterms_df = pd.read_csv("../../ontology_operations/data/GO_terms_biological_processes_merged_EBI_uniprot.csv", index_col=0)

In [6]:
goterms_df["gene_name"] = goterms_df["merged"].apply(lambda x: x.split(";"))
goterms_df = goterms_df.explode("gene_name")
goterms_df

,merged,len_merged,name,gene_name
GO:0000002,SLC25A33;PRSS15;MEF2;MGME1;PIF1;AAC1;C15orf20;...,21,mitochondrial genome maintenance,SLC25A33
GO:0000002,SLC25A33;PRSS15;MEF2;MGME1;PIF1;AAC1;C15orf20;...,21,mitochondrial genome maintenance,PRSS15
GO:0000002,SLC25A33;PRSS15;MEF2;MGME1;PIF1;AAC1;C15orf20;...,21,mitochondrial genome maintenance,MEF2
GO:0000002,SLC25A33;PRSS15;MEF2;MGME1;PIF1;AAC1;C15orf20;...,21,mitochondrial genome maintenance,MGME1
GO:0000002,SLC25A33;PRSS15;MEF2;MGME1;PIF1;AAC1;C15orf20;...,21,mitochondrial genome maintenance,PIF1
...,...,...,...,...
GO:2001272,OK/SW-cl.26;RNF78;RFPL1L;RPS3;RFPL1,5,positive regulation of cysteine-type endopepti...,RFPL1L
GO:2001272,OK/SW-cl.26;RNF78;RFPL1L;RPS3;RFPL1,5,positive regulation of cysteine-type endopepti...,RPS3
GO:2001272,OK/SW-cl.26;RNF78;RFPL1L;RPS3;RFPL1,5,positive regulation of cysteine-type endopepti...,RFPL1
GO:2001304,CYP4F3;LTB4H,2,lipoxin B4 metabolic process,CYP4F3


# Filter genes

In [36]:
gene_df = pd.DataFrame()
gene_df["gene_name"] = goterms_df.gene_name.unique()

In [39]:
go_bim = ot_bim.merge(gene_df, on="gene_name")

In [40]:
go_bim.gene_name.nunique() # some genes not mapped to go terms ..

16042

In [41]:
go_bim

,chr_id,position,ref_allele,alt_allele,chr_id_b37,position_b37,rs_id,most_severe_consequence,cadd,af,...,gene_id_prot_coding,chrom,snp,cm,pos,a1,a2,gene_id,gene_name,snp_bed_id
0,1,1169204,T,C,1,1104584,rs112695918,upstream_gene_variant,"{'phred': 0.61, 'raw': -0.223786}","{'gnomad_afr': 0.1370131366674349, 'gnomad_amr...",...,ENSG00000162571,1,rs112695918,0,1104584,T,C,ENSG00000162571,TTLL10,220
1,1,2038313,C,T,1,1969752,rs74372554,intergenic_variant,"{'phred': 0.309, 'raw': -0.324059}","{'gnomad_afr': 0.01078971533516988, 'gnomad_am...",...,ENSG00000067606,1,rs74372554,0,1969752,C,T,ENSG00000067606,PRKCZ,599
2,1,2194104,A,G,1,2125543,rs146607591,missense_variant,"{'phred': 4.241, 'raw': 0.098415}","{'gnomad_afr': 0.00034498620055197794, 'gnomad...",...,ENSG00000162585,1,rs146607591,0,2125543,A,G,ENSG00000162585,C1orf86,648
3,1,2592668,C,T,1,2524107,rs146100312,missense_variant,"{'phred': 23.9, 'raw': 3.118097}","{'gnomad_afr': 0.0, 'gnomad_amr': 0.0, 'gnomad...",...,ENSG00000157870,1,rs146100312,0,2524107,C,T,ENSG00000157870,FAM213B,838
4,1,3041036,T,C,1,2957600,rs12409277,intergenic_variant,"{'phred': 1.823, 'raw': -0.059547}","{'gnomad_afr': 0.13893357848770396, 'gnomad_am...",...,ENSG00000169717,1,rs12409277,0,2957600,T,C,ENSG00000169717,ACTRT2,958
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
644889,X,151609900,A,G,X,150778372,rs58448897,intron_variant,"{'phred': 3.523, 'raw': 0.055668}","{'gnomad_afr': 0.11482740118046861, 'gnomad_am...",...,ENSG00000166049,X,rs58448897,0,150778372,A,G,ENSG00000166049,PASD1,802181
644890,X,152014563,G,A,X,151183035,rs145754054,intergenic_variant,"{'phred': 2.456, 'raw': -0.011208}","{'gnomad_afr': 0.009298393913778529, 'gnomad_a...",...,ENSG00000102287,X,rs145754054,0,151183035,G,A,ENSG00000102287,GABRE,802336
644891,X,153459591,C,T,X,152725049,rs5945389,intron_variant,"{'phred': 2.968, 'raw': 0.022059}","{'gnomad_afr': 0.0032312925170068026, 'gnomad_...",...,ENSG00000183479,X,rs5945389,0,152725049,C,T,ENSG00000183479,TREX2,802547
644892,X,153781540,A,C,X,153046995,rs199915647,missense_variant,"{'phred': 22.5, 'raw': 2.480113}","{'gnomad_afr': 0.00017018379850238256, 'gnomad...",...,ENSG00000184343,X,rs199915647,0,153046995,A,C,ENSG00000184343,SRPK3,802648


In [42]:
gene_df = pd.DataFrame()
gene_df["gene_name"] = go_bim.gene_name.unique()
gene_df.reset_index(inplace=True)
gene_df.columns = ["gene_bed_id", "gene_name"]
gene_df

,gene_bed_id,gene_name
0,0,TTLL10
1,1,PRKCZ
2,2,C1orf86
3,3,FAM213B
4,4,ACTRT2
...,...,...
16037,16037,RPL18
16038,16038,HOXA3
16039,16039,TAF6
16040,16040,TPRN


In [43]:
go_bim = go_bim.merge(gene_df, on="gene_name")
go_bim

,chr_id,position,ref_allele,alt_allele,chr_id_b37,position_b37,rs_id,most_severe_consequence,cadd,af,...,chrom,snp,cm,pos,a1,a2,gene_id,gene_name,snp_bed_id,gene_bed_id
0,1,1169204,T,C,1,1104584,rs112695918,upstream_gene_variant,"{'phred': 0.61, 'raw': -0.223786}","{'gnomad_afr': 0.1370131366674349, 'gnomad_amr...",...,1,rs112695918,0,1104584,T,C,ENSG00000162571,TTLL10,220,0
1,1,2038313,C,T,1,1969752,rs74372554,intergenic_variant,"{'phred': 0.309, 'raw': -0.324059}","{'gnomad_afr': 0.01078971533516988, 'gnomad_am...",...,1,rs74372554,0,1969752,C,T,ENSG00000067606,PRKCZ,599,1
2,1,2194104,A,G,1,2125543,rs146607591,missense_variant,"{'phred': 4.241, 'raw': 0.098415}","{'gnomad_afr': 0.00034498620055197794, 'gnomad...",...,1,rs146607591,0,2125543,A,G,ENSG00000162585,C1orf86,648,2
3,1,2592668,C,T,1,2524107,rs146100312,missense_variant,"{'phred': 23.9, 'raw': 3.118097}","{'gnomad_afr': 0.0, 'gnomad_amr': 0.0, 'gnomad...",...,1,rs146100312,0,2524107,C,T,ENSG00000157870,FAM213B,838,3
4,1,3041036,T,C,1,2957600,rs12409277,intergenic_variant,"{'phred': 1.823, 'raw': -0.059547}","{'gnomad_afr': 0.13893357848770396, 'gnomad_am...",...,1,rs12409277,0,2957600,T,C,ENSG00000169717,ACTRT2,958,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
644889,X,151609900,A,G,X,150778372,rs58448897,intron_variant,"{'phred': 3.523, 'raw': 0.055668}","{'gnomad_afr': 0.11482740118046861, 'gnomad_am...",...,X,rs58448897,0,150778372,A,G,ENSG00000166049,PASD1,802181,9628
644890,X,152014563,G,A,X,151183035,rs145754054,intergenic_variant,"{'phred': 2.456, 'raw': -0.011208}","{'gnomad_afr': 0.009298393913778529, 'gnomad_a...",...,X,rs145754054,0,151183035,G,A,ENSG00000102287,GABRE,802336,7660
644891,X,153459591,C,T,X,152725049,rs5945389,intron_variant,"{'phred': 2.968, 'raw': 0.022059}","{'gnomad_afr': 0.0032312925170068026, 'gnomad_...",...,X,rs5945389,0,152725049,C,T,ENSG00000183479,TREX2,802547,2413
644892,X,153781540,A,C,X,153046995,rs199915647,missense_variant,"{'phred': 22.5, 'raw': 2.480113}","{'gnomad_afr': 0.00017018379850238256, 'gnomad...",...,X,rs199915647,0,153046995,A,C,ENSG00000184343,SRPK3,802648,12949


In [44]:
all_snp_ids = go_bim.snp_bed_id.unique() # list of all snp ids, enumerate and map to ids
snp_id_dict = dict(zip(all_snp_ids, range(len(all_snp_ids))))
# snp position in gene_level bed file
go_bim['snp_gbed_id'] = go_bim['snp_bed_id'].map(snp_id_dict)

In [47]:
go_bim[["snp", "gene_name", "snp_bed_id", "snp_gbed_id", "gene_bed_id"]]

,snp,gene_name,snp_bed_id,snp_gbed_id,gene_bed_id
0,rs112695918,TTLL10,220,0,0
1,rs74372554,PRKCZ,599,1,1
2,rs146607591,C1orf86,648,2,2
3,rs146100312,FAM213B,838,3,3
4,rs12409277,ACTRT2,958,4,4
...,...,...,...,...,...
644889,rs58448897,PASD1,802181,644889,9628
644890,rs145754054,GABRE,802336,644890,7660
644891,rs5945389,TREX2,802547,644891,2413
644892,rs199915647,SRPK3,802648,644892,12949


In [48]:
go_bim[["snp", "gene_name", "snp_bed_id", "snp_gbed_id", "gene_bed_id"]].to_csv("../data/go_biological_processes_bim.csv", index=False, sep="\t")

In [ ]:
go_bim.to_csv("../data/go_biological_processes_bim_full.csv", index=False, sep="\t")

# Prune network

In [7]:
import networkx as nx
import pickle

go_path = "../../ontology_operations/data/GO_biological_process.pickle"
G = pickle.load(open(go_path, 'rb'))

In [8]:
go_bim = pd.read_csv("../data/go_biological_processes_bim_full.csv", sep="\t")

In [9]:
merged_go = go_bim.merge(goterms_df.reset_index(), on="gene_name") # .... lot's of multiple connections

In [10]:
merged_go

,chr_id,position,ref_allele,alt_allele,chr_id_b37,position_b37,rs_id,most_severe_consequence,cadd,af,...,a2,gene_id,gene_name,snp_bed_id,gene_bed_id,snp_gbed_id,index,merged,len_merged,name
0,1,1169204,T,C,1,1104584,rs112695918,upstream_gene_variant,"{'phred': 0.61, 'raw': -0.223786}","{'gnomad_afr': 0.1370131366674349, 'gnomad_amr...",...,C,ENSG00000162571,TTLL10,220,0,0,GO:0018094,TTLL8;TTLL3;TTLL10,3,protein polyglycylation
1,1,2038313,C,T,1,1969752,rs74372554,intergenic_variant,"{'phred': 0.309, 'raw': -0.324059}","{'gnomad_afr': 0.01078971533516988, 'gnomad_am...",...,T,ENSG00000067606,PRKCZ,599,1,1,GO:0000226,C6orf97;DYNC1LI1;NF68;HAUS6;B7Z3Y3;LL5B;PARCC1...,346,microtubule cytoskeleton organization
2,1,2038313,C,T,1,1969752,rs74372554,intergenic_variant,"{'phred': 0.309, 'raw': -0.324059}","{'gnomad_afr': 0.01078971533516988, 'gnomad_am...",...,T,ENSG00000067606,PRKCZ,599,1,1,GO:0001954,DAG1;PLEKHA2;CCL25;CCR7;SCAP1;KIAA0457;CSF1;SC...,75,positive regulation of cell-matrix adhesion
3,1,2038313,C,T,1,1969752,rs74372554,intergenic_variant,"{'phred': 0.309, 'raw': -0.324059}","{'gnomad_afr': 0.01078971533516988, 'gnomad_am...",...,T,ENSG00000067606,PRKCZ,599,1,1,GO:0006468,STK22A;CDK13;SNRK;PRKACG;DAPK;ZAK;PSKH2;FLT2;D...,1149,protein phosphorylation
4,1,2038313,C,T,1,1969752,rs74372554,intergenic_variant,"{'phred': 0.309, 'raw': -0.324059}","{'gnomad_afr': 0.01078971533516988, 'gnomad_am...",...,T,ENSG00000067606,PRKCZ,599,1,1,GO:0006954,LY96;SCN11A;OLR1;C4A;ZAK;TNIP2;F2RL1;LYN;KCNJ8...,1106,inflammatory response
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5861945,X,154534419,G,A,X,153762634,rs5030868,missense_variant,"{'phred': 24.7, 'raw': 3.374046}","{'gnomad_afr': 0.0001721763085399449, 'gnomad_...",...,A,ENSG00000269335,IKBKG,802968,13478,644893,GO:0050852,PTPN6;UNQ744/PRO1472;MNAB;BT2.1;PS1;HLA-DP1B;M...,271,T cell receptor signaling pathway
5861946,X,154534419,G,A,X,153762634,rs5030868,missense_variant,"{'phred': 24.7, 'raw': 3.374046}","{'gnomad_afr': 0.0001721763085399449, 'gnomad_...",...,A,ENSG00000269335,IKBKG,802968,13478,644893,GO:0050862,TAPA1;CD81;CARD11;KCNN4;CCR7;RPS3;IKBKG;NFKB3;...,52,positive regulation of T cell receptor signali...
5861947,X,154534419,G,A,X,153762634,rs5030868,missense_variant,"{'phred': 24.7, 'raw': 3.374046}","{'gnomad_afr': 0.0001721763085399449, 'gnomad_...",...,A,ENSG00000269335,IKBKG,802968,13478,644893,GO:0051092,CARMA2;PIDD1;NFKB3;COP;CLAN1;TRIM62;HSPC187;IK...,322,positive regulation of NF-kappaB transcription...
5861948,X,154534419,G,A,X,153762634,rs5030868,missense_variant,"{'phred': 24.7, 'raw': 3.374046}","{'gnomad_afr': 0.0001721763085399449, 'gnomad_...",...,A,ENSG00000269335,IKBKG,802968,13478,644893,GO:0051650,RAB7;RAB11;IKBKG;FIP3;SNAPIN;TESK1;TCIRG1;ATP6...,15,establishment of vesicle localization


In [11]:
merged_go["index"].nunique()

11364

In [12]:
go_terms = merged_go["index"].unique()

In [13]:
go_terms = {i.replace(":", "_") for i in go_terms}
go_terms

{'GO_0060922',
 'GO_0006658',
 'GO_0097264',
 'GO_0061311',
 'GO_0035268',
 'GO_0030213',
 'GO_0021935',
 'GO_0021722',
 'GO_1902416',
 'GO_1903043',
 'GO_0032088',
 'GO_0090494',
 'GO_0046513',
 'GO_0097279',
 'GO_0031173',
 'GO_0098664',
 'GO_0050806',
 'GO_1902949',
 'GO_0072716',
 'GO_0097528',
 'GO_0042462',
 'GO_0034201',
 'GO_0006265',
 'GO_0070262',
 'GO_0061951',
 'GO_0035694',
 'GO_0008610',
 'GO_0046833',
 'GO_0016185',
 'GO_2000347',
 'GO_0060441',
 'GO_0043011',
 'GO_0048822',
 'GO_2000331',
 'GO_0060447',
 'GO_0034392',
 'GO_0032376',
 'GO_0045333',
 'GO_0001111',
 'GO_0140896',
 'GO_0002759',
 'GO_0002517',
 'GO_0055064',
 'GO_0002581',
 'GO_0106089',
 'GO_0051450',
 'GO_1901189',
 'GO_0035910',
 'GO_0042635',
 'GO_0034148',
 'GO_0010569',
 'GO_1904045',
 'GO_0034088',
 'GO_0070889',
 'GO_0009583',
 'GO_0065002',
 'GO_1904108',
 'GO_2000768',
 'GO_0019083',
 'GO_1990115',
 'GO_1990466',
 'GO_0009396',
 'GO_2000172',
 'GO_0044381',
 'GO_0046052',
 'GO_0061092',
 'GO_00435

In [14]:
print("Nodes before pruning:", len(G.nodes)) # 24k

Nodes before pruning: 24868


In [17]:
# prune network
def prune_tree(G, nodeset):
    nodes_to_check = list(G.nodes)
    while nodes_to_check:
        node = nodes_to_check.pop()
        if G.out_degree(node) == 0 and G.in_degree(node) > 0 and not node in nodeset: # if leaf is not in nodeset
            parents = list(G.predecessors(node))
            G.remove_node(node)
            nodes_to_check.extend(parents)

prune_tree(G, go_terms)

print("Number of nodes left", len(G.nodes))

Number of nodes left 23731


# Build full ontology

In [40]:
# Convert the pruned graph to a dataframe for output
pruned_edges = [(u, v) for u, v in G.edges]
ontology_df = pd.DataFrame(pruned_edges, columns=['parent', 'child'])
ontology_df["type"] = "default"
ontology_df

# append gene to go term df
tmp_df = merged_go[["index","gene_name"]].drop_duplicates()
tmp_df['index'] = tmp_df['index'].str.replace(":","_")
tmp_df.columns = ['parent', 'child']
tmp_df['type'] = 'gene'

# append snp to gene df
snp_df = go_bim[["gene_name","snp"]]
snp_df.columns = ['parent', 'child']
snp_df['type'] = 'snp'

# concatenate
ontology_df = pd.concat([ontology_df, tmp_df, snp_df])
ontology_df

/scratch/tmp.20794777.dgankin/ipykernel_4021084/3296272266.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  snp_df['type'] = 'snp'


,parent,child,type
0,GO_0000001,GO_0048311,default
1,GO_0000002,GO_0007005,default
2,GO_0000012,GO_0006281,default
3,GO_0000017,GO_0042946,default
4,GO_0000018,GO_0051052,default
...,...,...,...
644889,PASD1,rs58448897,snp
644890,GABRE,rs145754054,snp
644891,TREX2,rs5945389,snp
644892,SRPK3,rs199915647,snp


In [43]:
# Create a directed graph from the ontology data
OG = nx.DiGraph()

# Add edges to the graph
for index, row in ontology_df.iterrows():
    OG.add_edge(row['parent'], row['child'], type=row['type']) # only add gene if in known genes

print("Nodes in Graph:", len(OG.nodes))
print("Edges in Graph:", len(OG.edges))

#print(G.nodes)

# Step 2: Iteratively remove nodes that do not lead to a known gene
def prune_tree_(G):
    nodes_to_check = list(G.nodes)
    while nodes_to_check:
        node = nodes_to_check.pop()
        if G.out_degree(node) == 0 and G.in_degree(node) > 0 and "GO" in node:  # Check if it is a hanging node
            print("yas")
            parents = list(G.predecessors(node))
            G.remove_node(node)
            nodes_to_check.extend(parents)

prune_tree_(OG)

print("Number of nodes left", len(OG.nodes))

# Convert the pruned graph back to a dataframe for output
pruned_edges = [(u, v, OG[u][v]['type']) for u, v in OG.edges]
pruned_df = pd.DataFrame(pruned_edges, columns=['parent', 'child', 'type'])

# Save the pruned ontology
pruned_ontology_file = '../data/go_biological_processes.txt'
pruned_df.to_csv(pruned_ontology_file, sep='\t', index=False)

pruned_df.head()

Nodes in Graph: 683236
Edges in Graph: 792962
Number of nodes left 683236


,parent,child,type
0,GO_0000001,GO_0048311,default
1,GO_0048311,GO_0007005,default
2,GO_0048311,GO_0051646,default
3,GO_0048311,MEF2A,gene
4,GO_0048311,HAP1,gene
